In [4]:
# monthly_rebalance_run.py

import calendar
import datetime

from qstrader import settings
from qstrader.strategy.base import AbstractStrategy
from qstrader.position_sizer.rebalance import LiquidateRebalancePositionSizer
from qstrader.event import SignalEvent, EventType
from qstrader.compat import queue
from qstrader.trading_session import TradingSession


class MonthlyLiquidateRebalanceStrategy(AbstractStrategy):
    """
    A generic strategy that allows monthly rebalancing of a
    set of tickers, via full liquidation and dollar-weighting
    of new positions.

    Must be used in conjunction with the
    LiquidateRebalancePositionSizer object to work correctly.
    """
    def __init__(self, tickers, events_queue):
        self.tickers = tickers
        self.events_queue = events_queue
        self.tickers_invested = self._create_invested_list()

    def _end_of_month(self, cur_time):
        """
        Determine if the current day is at the end of the month.
        """
        cur_day = cur_time.day
        end_day = calendar.monthrange(cur_time.year, cur_time.month)[1]
        return cur_day == end_day

    def _create_invested_list(self):
        """
        Create a dictionary with each ticker as a key, with
        a boolean value depending upon whether the ticker has
        been "invested" yet. This is necessary to avoid sending
        a liquidation signal on the first allocation.
        """
        tickers_invested = {ticker: False for ticker in self.tickers}
        return tickers_invested

    def calculate_signals(self, event):
        """
        For a particular received BarEvent, determine whether
        it is the end of the month (for that bar) and generate
        a liquidation signal, as well as a purchase signal,
        for each ticker.
        """
        if (
            event.type in [EventType.BAR, EventType.TICK] and
            self._end_of_month(event.time)
        ):
            ticker = event.ticker
            if self.tickers_invested[ticker]:
                liquidate_signal = SignalEvent(ticker, "EXIT")
                self.events_queue.put(liquidate_signal)
            long_signal = SignalEvent(ticker, "BOT")
            self.events_queue.put(long_signal)
            self.tickers_invested[ticker] = True


def run_monthly_rebalance(
        tickers, ticker_weights, title, 
        start_date, end_date, initial_equity
    ):
    testing = False
    config = settings.from_file(
        settings.DEFAULT_CONFIG_FILENAME, testing
    )

    # Use the Monthly Liquidate And Rebalance strategy
    events_queue = queue.Queue()
    strategy = MonthlyLiquidateRebalanceStrategy(
        tickers, events_queue
    )

    # Use the liquidate and rebalance position sizer
    # with prespecified ticker weights
    position_sizer = LiquidateRebalancePositionSizer(
        ticker_weights
    )

    # Set up the backtest
    backtest = TradingSession(
        config, strategy, tickers,
        initial_equity, start_date, end_date,
        events_queue, position_sizer=position_sizer,
        title=[title], benchmark=tickers[0],
    )
    results = backtest.start_trading(testing=testing)
    return results


In [ ]:
# equities_bonds_60_40_etf_portfolio_backtest.py

import datetime

from qstrader import settings
from monthly_rebalance_run import run_monthly_rebalance


if __name__ == "__main__":
    tickers = ["SPY", "AGG"]
    ticker_weights = {
        "SPY": 0.6,
        "AGG": 0.4,
    }
    run_monthly_rebalance(
        tickers, ticker_weights, 
        title="US Equities/Bonds 60/40 Mix ETF Strategy",
        start_date=datetime.datetime(2003, 9, 29), 
        end_date=datetime.datetime(2016, 10, 12),
        initial_equity=500000.00
    )


QSTRADER中需要重写的class

subclass of the AbstractStrategy base class called MonthlyLiquidateRebalanceStrategy.

subclass of the AbstractPositionSizer base class found within the QSTrader
called LiquidateRebalancePositionSizer.

In [ ]:
# strategic_weight_etf_portfolio_backtest.py

import datetime

from qstrader import settings
from monthly_rebalance_run import run_monthly_rebalance


if __name__ == "__main__":
    tickers = [
        "SPY", "IJS", "EFA", "EEM", 
        "AGG", "JNK", "DJP", "RWR"
    ]
    ticker_weights = {
        "SPY": 0.25,
        "IJS": 0.05,
        "EFA": 0.20,
        "EEM": 0.05,
        "AGG": 0.20,
        "JNK": 0.05,
        "DJP": 0.10,
        "RWR": 0.10 
    }
    run_monthly_rebalance(
        tickers, ticker_weights, 
        title="Strategic Weight ETF Strategy",
        start_date=datetime.datetime(2007, 12, 4), 
        end_date=datetime.datetime(2016, 10, 12),
        initial_equity=500000.00
    )


In [ ]:
# equal_weight_etf_portfolio_backtest.py

import datetime

from qstrader import settings
from monthly_rebalance_run import run_monthly_rebalance


if __name__ == "__main__":
    tickers = [
        "SPY", "IJS", "EFA", "EEM", 
        "AGG", "JNK", "DJP", "RWR"
    ]
    ticker_weights = {
        "SPY": 0.125,
        "IJS": 0.125,
        "EFA": 0.125,
        "EEM": 0.125,
        "AGG": 0.125,
        "JNK": 0.125,
        "DJP": 0.125,
        "RWR": 0.125 
    }
    run_monthly_rebalance(
        tickers, ticker_weights, 
        title="Equal Weight ETF Strategy",
        start_date=datetime.datetime(2007, 12, 4), 
        end_date=datetime.datetime(2016, 10, 12),
        initial_equity=500000.00
    )


# pair trading

The implementation of the strategy is similar to other QSTrader strategies. It involves the
creation of a subclass of AbstractStrategy in the coint_bollinger_strategy.py file. This
class is then used by the coint_bollinger_backtest.py file to actually simulate the backtest.

In [3]:
# coint_bollinger_strategy.py
from __future__ import print_function
from collections import deque
from math import floor
import numpy as np
from qstrader.price_parser import PriceParser
from qstrader.event import (SignalEvent, EventType)
from qstrader.strategy.base import AbstractStrategy

class CointegrationBollingerBandsStrategy(AbstractStrategy):
    """
    Requires:
    tickers - The list of ticker symbols
    events_queue - A handle to the system events queue
    lookback - Lookback period for moving avg and moving std
    weights - The weight vector describing
    a "unit" of the portfolio
    entry_z - The z-score trade entry threshold
    exit_z - The z-score trade exit threshold
    base_quantity - Number of "units" of the portfolio
    to be traded
    """
    def __init__(self, tickers, events_queue,lookback, weights, entry_z, exit_z,base_quantity):
        self.tickers = tickers
        self.events_queue = events_queue
        self.lookback = lookback
        self.weights = weights
        self.entry_z = entry_z
        self.exit_z = exit_z
        self.qty = base_quantity
        self.time = None
        self.latest_prices = np.full(len(self.tickers), -1.0)
        self.port_mkt_val = deque(maxlen=self.lookback)
        self.invested = None
        self.bars_elapsed = 0
    def _set_correct_time_and_price(self, event):
        """
        Sets the correct price and event time for prices
        that arrive out of order in the events queue.
        """
        # Set the first instance of time
        if self.time is None:
            self.time = event.time
        # Set the correct latest prices depending upon
        # order of arrival of market bar event
        price = event.adj_close_price/float(PriceParser.PRICE_MULTIPLIER)
        if event.time == self.time:
            for i in range(0, len(self.tickers)):
                if event.ticker == self.tickers[i]:
                    self.latest_prices[i] = price
        else:
            self.time = event.time
            self.bars_elapsed += 1
            self.latest_prices = np.full(len(self.tickers), -1.0)
            for i in range(0, len(self.tickers)):
                if event.ticker == self.tickers[i]:
                    self.latest_prices[i] = price
    def go_long_units(self):
        """
        Go long the appropriate number of "units" of the
        portfolio to open a new position or to close out
        a short position.
        """
        for i, ticker in enumerate(self.tickers):
            if self.weights[i] < 0.0:
                self.events_queue.put(SignalEvent(ticker, "SLD",int(floor(-1.0*self.qty*self.weights[i]))))
            else:
                self.events_queue.put(SignalEvent(ticker, "BOT",int(floor(self.qty*self.weights[i]))))
    def go_short_units(self):
        """
        Go short the appropriate number of "units" of the
        portfolio to open a new position or to close out
        a long position.
        """
        for i, ticker in enumerate(self.tickers):
            if self.weights[i] < 0.0:
                self.events_queue.put(SignalEvent(ticker, "BOT",int(floor(-1.0*self.qty*self.weights[i]))))
            else:
                self.events_queue.put(SignalEvent(ticker, "SLD",int(floor(self.qty*self.weights[i]))))
    def zscore_trade(self, zscore, event):
        """
        Determine whether to trade if the entry or exit zscore
        threshold has been exceeded.
        """
        # If we’re not in the market...
        if self.invested is None:
            if zscore < -self.entry_z:
                # Long Entry
                print("LONG: %s" % event.time)
                self.go_long_units()
                self.invested = "long"
            elif zscore > self.entry_z:
                # Short Entry
                print("SHORT: %s" % event.time)
                self.go_short_units()
                self.invested = "short"
        # If we are in the market...
        if self.invested is not None:
            if self.invested == "long" and zscore >= -self.exit_z:
                print("CLOSING LONG: %s" % event.time)
                self.go_short_units()
                self.invested = None
            elif self.invested == "short" and zscore <= self.exit_z:
                print("CLOSING SHORT: %s" % event.time)
                self.go_long_units()
                self.invested = None
    def calculate_signals(self, event):
        """
        Calculate the signals for the strategy.
        """
        if event.type == EventType.BAR:
            self._set_correct_time_and_price(event)
            # Only trade if we have all prices
            if all(self.latest_prices > -1.0):
                # Calculate portfolio market value via dot product
                # of ETF prices with portfolio weights
                self.port_mkt_val.append(np.dot(self.latest_prices, self.weights))
                # If there is enough data to form a full lookback
                # window, then calculate zscore and carry out
                # respective trades if thresholds are exceeded
                if self.bars_elapsed > self.lookback:
                    zscore = (self.port_mkt_val[-1] - np.mean(self.port_mkt_val)) / np.std(self.port_mkt_val)
                    self.zscore_trade(zscore, event)

In [ ]:
# coint_bollinger_backtest.py

import datetime

import numpy as np

from qstrader import settings
from qstrader.strategy.base import AbstractStrategy
from qstrader.position_sizer.fixed import FixedPositionSizer
from qstrader.event import SignalEvent, EventType
from qstrader.compat import queue
from qstrader.trading_session import TradingSession

from coint_bollinger_strategy import CointegrationBollingerBandsStrategy


def run(config, testing, tickers, filename):
    # Backtest information
    title = [
        'Aluminium Smelting Strategy - ARNC/UNG'
    ]
    initial_equity = 500000.0
    start_date = datetime.datetime(2013, 6, 8)
    end_date = datetime.datetime(2018, 6, 8)

    # Use the Cointegration Bollinger Bands trading strategy
    events_queue = queue.Queue()
    weights = np.array([1.0, -1.213])
    lookback = 15
    entry_z = 1.5
    exit_z = 0.5
    base_quantity = 10000
    strategy = CointegrationBollingerBandsStrategy(
        tickers, events_queue, 
        lookback, weights, 
        entry_z, exit_z, base_quantity
    )

    # Set the position size to use a 
    # fixed base quantity of shares
    position_sizer = FixedPositionSizer(
        default_quantity=base_quantity
    )

    # Set up the backtest
    backtest = TradingSession(
        config, strategy, tickers,
        initial_equity, start_date, end_date,
        events_queue, title=title,
        position_sizer=position_sizer
    )
    results = backtest.start_trading(testing=testing)
    return results


if __name__ == "__main__":
    # Configuration data
    testing = False
    config = settings.from_file(
        settings.DEFAULT_CONFIG_FILENAME, testing
    )
    tickers = ["ARNC", "UNG"]
    filename = None
    run(config, testing, tickers, filename)


# kalman filter

In [7]:
# kalman_qstrader_strategy.py
from math import floor
import numpy as np
from qstrader.price_parser import PriceParser
from qstrader.event import (SignalEvent, EventType)
from qstrader.strategy.base import AbstractStrategy
class KalmanPairsTradingStrategy(AbstractStrategy):
    """
    Requires:
    tickers - The list of ticker symbols
    events_queue - A handle to the system events queue
    """
    def __init__(self, tickers, events_queue):
        self.tickers = tickers
        self.events_queue = events_queue
        self.time = None
        self.latest_prices = np.array([-1.0, -1.0])
        self.invested = None
        self.delta = 1e-4
        self.wt = self.delta / (1 - self.delta) * np.eye(2)
        self.vt = 1e-3
        self.theta = np.zeros(2)
        self.P = np.zeros((2, 2))
        self.R = None
        self.days = 0
        self.qty = 2000
        self.cur_hedge_qty = self.qty
    def _set_correct_time_and_price(self, event):
        """
        Sets the correct price and event time for prices
        that arrive out of order in the events queue.
        """
        # Set the first instance of time
        if self.time is None:
            self.time = event.time
        # Set the correct latest prices depending upon
        # order of arrival of market bar event
        price = event.adj_close_price/float(PriceParser.PRICE_MULTIPLIER)
        if event.time == self.time:
            if event.ticker == self.tickers[0]:
                self.latest_prices[0] = price
            else:
                self.latest_prices[1] = price
        else:
            self.time = event.time
            self.days += 1
            self.latest_prices = np.array([-1.0, -1.0])
            if event.ticker == self.tickers[0]:
                self.latest_prices[0] = price
            else:
                self.latest_prices[1] = price
    def calculate_signals(self, event):
        """
        Calculate the Kalman Filter strategy.
        """
        if event.type == EventType.BAR:
            self._set_correct_time_and_price(event)
            # Only trade if we have both observations
            if all(self.latest_prices > -1.0):
                # Create the observation matrix of the latest prices
                # of TLT and the intercept value (1.0) as well as the
                # scalar value of the latest price from IEI
                F = np.asarray([self.latest_prices[0], 1.0]).reshape((1, 2))
                y = self.latest_prices[1]
                # The prior value of the states \theta_t is
                # distributed as a multivariate Gaussian with
                # mean a_t and variance-covariance R_tif self.R is not None:
                if self.R is not None:
                    self.R = self.C + self.wt
                else:
                    self.R = np.zeros((2, 2))
                # Calculate the Kalman Filter update
                # ----------------------------------
                # Calculate prediction of new observation
                # as well as forecast error of that prediction
                yhat = F.dot(self.theta)
                et = y - yhat
                # Q_t is the variance of the prediction of
                # observations and hence \sqrt{Q_t} is the
                # standard deviation of the predictions
                Qt = F.dot(self.R).dot(F.T) + self.vt
                sqrt_Qt = np.sqrt(Qt)
                # The posterior value of the states \theta_t is
                # distributed as a multivariate Gaussian with mean
                # m_t and variance-covariance C_t
                At = self.R.dot(F.T) / Qt
                self.theta = self.theta + At.flatten() * et
                self.C = self.R - At * F.dot(self.R)
                # Only trade if days is greater than a "burn in" period
                if self.days > 1:
                    # If we’re not in the market...
                    if self.invested is None:
                        if et < -sqrt_Qt:
                            # Long Entry
                            print("LONG: %s" % event.time)
                            self.cur_hedge_qty = int(floor(self.qty*self.theta[0]))
                            self.events_queue.put(SignalEvent(self.tickers[1],"BOT", self.qty))
                            self.events_queue.put(SignalEvent(self.tickers[0],"SLD", self.cur_hedge_qty))
                            self.invested = "long"
                        elif et > sqrt_Qt:
                            # Short Entry
                            print("SHORT: %s" % event.time)
                            self.cur_hedge_qty = int(floor(self.qty*self.theta[0]))
                            self.events_queue.put(SignalEvent(self.tickers[1], "SLD", self.qty))
                            self.events_queue.put(SignalEvent(self.tickers[0], "BOT",self.cur_hedge_qty))
                            self.invested = "short"
                    # If we are in the market...
                    if self.invested is not None:
                        if self.invested == "long" and et > -sqrt_Qt:
                            print("CLOSING LONG: %s" % event.time)
                            self.events_queue.put(SignalEvent(self.tickers[1],"SLD", self.qty))
                            self.events_queue.put(SignalEvent(self.tickers[0],"BOT", self.cur_hedge_qty))
                            self.invested = None
                        elif self.invested == "short" and et < sqrt_Qt:
                            print("CLOSING SHORT: %s" % event.time)
                            self.events_queue.put(SignalEvent(self.tickers[1],"BOT", self.qty))
                            self.events_queue.put(SignalEvent(self.tickers[0],"SLD", self.cur_hedge_qty))
                            self.invested = None

In [ ]:
# kalman_qstrader_backtest.py

import datetime

from qstrader import settings
from qstrader.strategy.base import AbstractStrategy
from qstrader.position_sizer.naive import NaivePositionSizer
from qstrader.event import SignalEvent, EventType
from qstrader.compat import queue
from qstrader.trading_session import TradingSession

from kalman_qstrader_strategy import KalmanPairsTradingStrategy


def run(config, testing, tickers, filename):
    # Backtest information
    title = [
        'Kalman Filter Pairs Trade on TLT/IEI'
    ]
    initial_equity = 100000.0
    start_date = datetime.datetime(2013, 6, 7)
    end_date = datetime.datetime(2018, 6, 7)

    # Use the KalmanPairsTrading Strategy
    events_queue = queue.Queue()
    strategy = KalmanPairsTradingStrategy(
        tickers, events_queue
    )

    # Use the Naive Position Sizer where
    # suggested quantities are followed
    position_sizer = NaivePositionSizer()

    # Set up the backtest
    backtest = TradingSession(
        config, strategy, tickers,
        initial_equity, start_date, end_date,
        events_queue, title=title,
        position_sizer=position_sizer
    )
    results = backtest.start_trading(testing=testing)
    return results


if __name__ == "__main__":
    # Configuration data
    testing = False
    config = settings.from_file(
        settings.DEFAULT_CONFIG_FILENAME, testing
    )
    tickers = ["TLT", "IEI"]
    filename = None
    run(config, testing, tickers, filename)


# Intraday Machine Learning

In [106]:
# intraday_ml_model_fit.py
import datetime
import numpy as np
import pandas as pd
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier)
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
def create_up_down_dataframe(csv_filepath,lookback_minutes=30,lookforward_minutes=5,up_down_factor=2.0,
percent_factor=0.01,start=None, end=None):
    """
    Creates a Pandas DataFrame that imports and calculates
    the percentage returns of an intraday OLHC ticker from disk.
    ’lookback_minutes’ of prior returns are stored to create
    a feature vector, while ’lookforward_minutes’ are used to
    ascertain how far in the future to predict across.
    The actual prediction is to determine whether a ticker
    moves up by at least ’up_down_factor’ x ’percent_factor’,
    while not dropping below ’percent_factor’ in the same period.
    i.e. Does the stock move up 1% in a minute and not down by 0.5%?
    The DataFrame will consist of ’lookback_minutes’ columns for feature
    vectors and one column for whether the stock adheres to the "up/down"
    rule, which is 1 if True or 0 if False for each minute.
    """
    ts = pd.read_csv(csv_filepath,names=["Date", "Open",  "High","Low","Close", "Volume", "Adj Close"], header=0,parse_dates=True)# Filter on start/end dates
    ts['Date'] = pd.to_datetime(ts['Date'])
    ts=ts.set_index('Date')
    if start is not None:
        ts = ts[ts.index >= start]
    if end is not None:
        ts = ts[ts.index <= end]
    # Drop the non-essential columns
    ts.drop(["Open",  "High","Low","Volume", "Adj Close"],axis=1, inplace=True)
    # Create the lookback and lookforward shifts
    for i in range(0, lookback_minutes):
        ts["Lookback%s" % str(i+1)] = ts["Close"].shift(i+1)
    for i in range(0, lookforward_minutes):
        ts["Lookforward%s" % str(i+1)] = ts["Close"].shift(-(i+1))
    ts.dropna(inplace=True)
    # Adjust all of these values to be percentage returns
    #print(ts.head())
    ts["Lookback0"] =np.concatenate([[np.nan],(ts['Close'][1:].values-ts['Close'][:-1].values)/ts['Close'][:-1].values*100.0]) 
    for i in range(0, lookback_minutes):
        ts["Lookback%s" % str(i+1)] = np.concatenate([[np.nan],(ts["Lookback%s" % str(i+1)][1:].values-ts["Lookback%s" % str(i+1)][:-1].values)/ts["Lookback%s" % str(i+1)][:-1].values*100.0]) 
    for i in range(0, lookforward_minutes):
        ts["Lookforward%s" % str(i+1)] = np.concatenate([[np.nan],(ts["Lookforward%s" % str(i+1)][1:].values-ts["Lookforward%s" % str(i+1)][:-1].values)/ts["Lookforward%s" % str(i+1)][:-1].values*100.0]) 
    ts.dropna(inplace=True)
    # Determine if the stock has gone up at least by
    # ’up_down_factor’ x ’percent_factor’ and down no more
    # then ’percent_factor’
    up = up_down_factor*percent_factor
    down = percent_factor
    # Create the list of True/False entries for each date
    # as to whether the up/down logic is true
    down_cols = [ts["Lookforward%s" % str(i+1)] > -down for i in range(0, lookforward_minutes)]
    up_cols = [ts["Lookforward%s" % str(i+1)] > up for i in range(0, lookforward_minutes)]
    # Carry out the bitwise and, as well as bitwise or
    # for the down and up logic
    down_tot = down_cols[0]
    for c in down_cols[1:]:
        down_tot = down_tot & c
    up_tot = up_cols[0]
    for c in up_cols[1:]:
        up_tot = up_tot | c
    #ts["UpDown"] = down_tot & up_tot
    ts["UpDown"] = np.sign(ts["Lookforward1"])
    # Convert True/False into 1 and 0
    ts["UpDown"] = ts["UpDown"].astype(int)
    ts["UpDown"].replace(to_replace=0, value=-1, inplace=True)
    return ts
if __name__ == "__main__":
    random_state = 42
    n_estimators = 400
    n_jobs = 1
    csv_filepath = "/Users/sheng/data/SPY.csv"
    lookback_minutes = 30
    lookforward_minutes = 5
    print("Importing and creating CSV DataFrame...")
    start_date = datetime.datetime(2007, 11, 8)
    end_date = datetime.datetime(2012, 12, 31)
    ts = create_up_down_dataframe(csv_filepath,lookback_minutes=lookback_minutes,
                                  lookforward_minutes=lookforward_minutes,
                                  start=start_date, end=end_date)
    # Use the first five daily lags of AREX closing prices
    print("Preprocessing data...")
    X = ts[["Lookback%s" % str(i)for i in range(0, 5)]]
    y = ts["UpDown"]
    # Use the training-testing split with 70% of data in the
    # training data with the remaining 30% of data in the testing
    print("Creating train/test split of data...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    print("Fitting classifier model...")
    model = RandomForestClassifier(n_estimators=n_estimators,n_jobs=n_jobs,random_state=random_state,max_depth=10)
    model.fit(X_train, y_train)
    print("Outputting metrics...")
    print("Hit-Rate: %s" % model.score(X_test, y_test))
    print("%s\n" % confusion_matrix(model.predict(X_test), y_test))
    print("Pickling model...")
    joblib.dump(model, '/Users/sheng/data/ml_model_rf.pkl')

Importing and creating CSV DataFrame...
Preprocessing data...
Creating train/test split of data...
Fitting classifier model...
Outputting metrics...
Hit-Rate: 0.5555555555555556
[[ 65  64]
 [104 145]]

Pickling model...


In [110]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
# intraday_ml_strategy.py

import numpy as np
import pandas as pd
from sklearn.externals import joblib

from qstrader.price_parser import PriceParser
from qstrader.event import (SignalEvent, EventType)
from qstrader.strategy.base import AbstractStrategy


class IntradayMachineLearningPredictionStrategy(AbstractStrategy):
    """
    Requires:
    tickers - The list of ticker symbols
    events_queue - A handle to the system events queue
    """
    def __init__(
        self, tickers, events_queue, 
        model_pickle_file, lags=5
    ):
        self.tickers = tickers
        self.events_queue = events_queue
        self.model_pickle_file = model_pickle_file
        self.lags = lags

        self.invested = False
        self.cur_prices = np.zeros(self.lags+1)
        self.cur_returns = np.zeros(self.lags)
        self.minutes = 0
        self.qty = 10000
        #load the learnt model from fit.py
        self.model = joblib.load(model_pickle_file)

    def _update_current_returns(self, event):
        """
        Updates the array of current returns "features"
        used by the machine learning model for prediction.
        """
        # Adjust the feature vector to move all lags by one
        # and then recalculate the returns
        for i, f in reversed(list(enumerate(self.cur_prices))):
            if i > 0:
                self.cur_prices[i] = self.cur_prices[i-1]
            else:
                self.cur_prices[i] = event.close_price/float(
                    PriceParser.PRICE_MULTIPLIER
                )
        if self.minutes > (self.lags + 1):
            for i in range(0, self.lags):
                self.cur_returns[i] = ((
                    self.cur_prices[i]/self.cur_prices[i+1]
                )-1.0)*100.0

    def calculate_signals(self, event):
        """
        Calculate the intraday machine learning 
        prediction strategy.
        """
        if event.type == EventType.BAR:
            self._update_current_returns(event)
            self.minutes += 1
            # Allow enough time to pass to populate the 
            # returns feature vector
            if self.minutes > (self.lags + 2):
                pred = self.model.predict(self.cur_returns.reshape((1, -1)))[0]
                # Long only strategy
                if not self.invested and pred == 1:
                    print("LONG: %s" % event.time)
                    self.events_queue.put(
                        SignalEvent(self.tickers[0], "BOT", self.qty)
                    )
                    self.invested = True
                if self.invested and pred == -1:
                    print("CLOSING LONG: %s" % event.time)
                    self.events_queue.put(
                        SignalEvent(self.tickers[0], "SLD", self.qty)
                    )
                    self.invested = False


In [ ]:
# kalman_qstrader_backtest.py

import datetime

from qstrader import settings
from qstrader.strategy.base import AbstractStrategy
from qstrader.position_sizer.naive import NaivePositionSizer
from qstrader.event import SignalEvent, EventType
from qstrader.compat import queue
from qstrader.trading_session import TradingSession

from kalman_qstrader_strategy import KalmanPairsTradingStrategy


def run(config, testing, tickers, filename):
    # Backtest information
    title = [
        'Kalman Filter Pairs Trade on TLT/IEI'
    ]
    initial_equity = 100000.0
    start_date = datetime.datetime(2013, 6, 7)
    end_date = datetime.datetime(2018, 6, 7)

    # Use the KalmanPairsTrading Strategy
    events_queue = queue.Queue()
    strategy = KalmanPairsTradingStrategy(
        tickers, events_queue
    )

    # Use the Naive Position Sizer where
    # suggested quantities are followed
    position_sizer = NaivePositionSizer()

    # Set up the backtest
    backtest = TradingSession(
        config, strategy, tickers,
        initial_equity, start_date, end_date,
        events_queue, title=title,
        position_sizer=position_sizer
    )
    results = backtest.start_trading(testing=testing)
    return results


if __name__ == "__main__":
    # Configuration data
    testing = False
    config = settings.from_file(
        settings.DEFAULT_CONFIG_FILENAME, testing
    )
    tickers = ["TLT", "IEI"]
    filename = None
    run(config, testing, tickers, filename)


In [ ]:
#clean for ARIMA+GARCH R code
if __name__ == "__main__":
    # Open the forecasts CSV file and read in the lines
    forecasts = open("forecasts.csv", "r").readlines()
    # Run through the list and lag the forecasts by one
    old_value = 1
    new_list = []
    for f in forecasts[1:]:
        strpf = f.replace('"’,'').strip()
        new_str = "%s,%s\n" % (strpf, old_value)
        newspl = new_str.strip().split(",")
        final_str = "%s,%s\n" % (newspl[0], newspl[2])
        final_str = final_str.replace('"','')
        old_value = f.strip().split(',')[1]
        new_list.append(final_str)

    # Output the updated forecasts CSV file
    out = open("forecasts_new.csv", "w")
    for n in new_list:
        out.write(n)